# ADS Project 2022 Group 7:  Image Classification with Transfer Learning

## Setup

In [ ]:
##ONLY RELEVANT FOR AWS SAGEMAKER: installing mechanicalsoup
!pip install mechanicalsoup

In [ ]:
##ONLY RELEVANT FOR AWS SAGEMAKER: installing wget
!pip install wget

In [ ]:
##ONLY RELEVANT FOR AWS SAGEMAKER: installing tensorflow
!pip install tensorflow

In [ ]:
#ONLY RELEVANT FOR AWS SAGEMAKER: installing kaggle
!pip install kaggle

In [ ]:
#stuff for web scraping
import mechanicalsoup
import wget

#stuff to handle files and folders
import os
import glob
import shutil

#other stuff
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import time

## Data

In [ ]:
#ONLY RELEVANT FOR AWS SAGEMAKER: Kaggle secret
#Step 1: Upload kaggle secret to jupyter notebook, to the same folder as this notebook is in. 
#Step 2: Uncomment and run the line of code below to move kaggle secret to the right location.
#mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle

In [ ]:
#connect to kaggle api
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
#download dataset from kaggle using api
api.dataset_download_files('shaunthesheep/microsoft-catsvsdogs-dataset', path='./')

In [ ]:
#unzip the zip file
import zipfile
with zipfile.ZipFile('microsoft-catsvsdogs-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('catsvsdogs')

In [ ]:
#resize a subset of images of dogs
dst_dir = 'catsvsdogs/resized/dogs'
os.makedirs(dst_dir, exist_ok=True)

files = glob.glob('./catsvsdogs/PetImages/Dog/*.jpg')

i = 0

for f in files: 
    i = i + 1
    try:
        img = Image.open(f)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img_resize = img.resize((192, 192))
        root, ext = os.path.splitext(f)
        basename = os.path.basename(root)
        img_resize.save(os.path.join(dst_dir, basename + ext))
    except (IOError) as e: #in some instances there were some corrupted files in the zip file.
        print ('Bad file:', f)
        pass
    if i == 3000:
        break

In [ ]:
#Resize a subset of images of cats 
dst_dir = 'catsvsdogs/resized/cats'
os.makedirs(dst_dir, exist_ok=True)

files = glob.glob('./catsvsdogs/PetImages/Cat/*.jpg')

i = 0

for f in files:
    i = i + 1
    try:
        img = Image.open(f)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img_resize = img.resize((192, 192))
        root, ext = os.path.splitext(f)
        basename = os.path.basename(root)
        img_resize.save(os.path.join(dst_dir, basename + ext))
    except (IOError) as e: #in some instances there were some corrupted files in the zip file.
        print ('Bad file:', f)
        pass
    if i == 3000:
        break

In [ ]:
#making sure the folders were created correctly
data_dir = os.path.join(os.curdir, 'catsvsdogs/resized')
os.listdir(data_dir)

In [ ]:
#OPTIONAL: Delete original unziped data

shutil.rmtree(os.path.join(os.curdir, 'catsvsdogs/PetImages'))

## Image Classification 

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (192, 192)

#shuffle immer mit seed, sonst kann man nicht das gleiche Dataset in Valdiation & Train verwenden.

train_dataset = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            shuffle=True,
                                                            seed=999,
                                                            validation_split=0.20,
                                                            subset = "training")

validation_dataset = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            shuffle=True,
                                                            seed=999,
                                                            validation_split=0.20,
                                                            subset = "validation")

In [ ]:
class_names = train_dataset.class_names

#plt.figure(figsize=(10, 10))
#for images, labels in train_dataset.take(1):
#  for i in range(9):
#    ax = plt.subplot(3, 3, i + 1)
#    plt.imshow(images[i].numpy().astype("uint8"))
#    plt.title(class_names[labels[i]])
#    plt.axis("off")

### Creation of test set by splitting the validation set into batches.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 10)
validation_dataset = validation_dataset.skip(val_batches // 10)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [ ]:
#Buffered prefetching
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Use data augmentation

In [ ]:
#To reduce overfitting, pictures are flipped and rotated

with tf.device('/cpu:0'):
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.RandomFlip('horizontal'),
      tf.keras.layers.RandomRotation(0.2),
    ])

In [ ]:
#test effect of augmentation

with tf.device('/cpu:0'):
    for image, _ in train_dataset.take(1):
      plt.figure(figsize=(10, 10))
      first_image = image[0]
      for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

### Choose Transfer Learning Model 

The following pre-trained model is used as base model for this image classifier project: **MobileNetV2** developed by Google (dowloaded via  `tf.keras.applications.MobileNetV2` ). 


In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
#rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

#OPEN QUESTION

#Wäre das noch nötig?

We saw in the lesson that, wen doing trasnfer learning, we don't use all the architecture but only the feature extraction.
The very last classification layer (on "top", as most diagrams of machine learning models go from bottom to top) is not very useful. Instead, you will follow the common practice to depend on the very last layer before the flatten operation. This layer is called the "bottleneck layer". The bottleneck layer features retain more generality as compared to the final/top layer.

First, instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the **include_top=False** argument, you load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [ ]:
# Create the base model from the pre-trained model MobileNet V2

IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

This feature extractor converts each `192x192x3` image into a `6x6x1280` block of features. Let's see what it does to an example batch of images:

In [ ]:
#image_batch, label_batch = next(iter(train_dataset))
#feature_batch = base_model(image_batch)
#print(feature_batch.shape)

### a. Freeze weights, use the feature extraction as it is, attach a new classification head
In this step, you will freeze the convolutional base created from the previous step and to use as a feature extractor. Additionally, you add a classifier on top of it and train the top-level classifier.

It is important to freeze the convolutional base before you compile and train the model. Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training. MobileNet V2 has many layers, so setting the entire model's `trainable` flag to False will freeze all of them.

In [ ]:
#freeze convolutional base before compiling and training the model.
base_model.trainable = False

In [ ]:
base_model.summary()

Let's now combine this model with a classification head, meaning some fully connected layers (tf.keras.layers.Dense). 

Build a model by chaining together the data augmentation, preproccesing, `base_model` and classification layers (note that the syntax is slighly different, we are using the [Keras Functional API]).



Apply a `tf.keras.layers.Dense` layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a `logit`, or a raw prediction value. Positive numbers predict class 1, negative numbers predict class 0.

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False) #We need to set `training=False` as our model contains a `BatchNormalization` layer. More explanation here https://www.tensorflow.org/guide/keras/transfer_learning
x = tf.keras.layers.GlobalAveragePooling2D()(x) #this layer has the same purpose as tf.keras.layers.Flatten(). Need to connect something 2D to something 1D. Ask if you want to know more :)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) 
model = tf.keras.Model(inputs, outputs)


Compile the model before training it. Since there are two classes, use the `tf.keras.losses.BinaryCrossentropy` loss with `from_logits=True` since the model provides a linear output.

In [ ]:
base_learning_rate = 0.1
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Above you see how many parameters you are actually training, and how many are frozen.

### Train the model



In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

In [ ]:
#plot learning curves of training and validation accuracy / loss.

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Note: If you are wondering why the validation metrics are better than the training metrics, the main factor is because layers like `tf.keras.layers.BatchNormalization` and `tf.keras.layers.Dropout` affect accuracy during training. They are turned off when calculating validation loss.

To a lesser extent, it is also because training metrics report the average for an epoch, while validation metrics are evaluated after the epoch, so validation metrics see a model that has trained slightly longer.

### Evaluation and prediction

Finaly you can verify the performance of the model on new data using test set.

In [ ]:
#Verify model performance on test set.
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Apply model with pictures obtained through web scraping

In [ ]:
#create browser object and pass google images url to it.
browser = mechanicalsoup.StatefulBrowser()
url = 'https://www.google.ch/imghp?hl=en&ogbl'
    
browser.open(url)
print(browser.get_url())

In [ ]:
#Find pictures of cats...
browser.get_current_page()
browser.select_form()
#browser.get_current_form().print_summary()

search_term = 'Katze'

browser['q'] = search_term
browser.launch_browser()
response = browser.submit_selected()
new_url = browser.get_url()
browser.open(new_url)
page = browser.get_current_page()
all_images = page.find_all('img')

#... and save them into a list.
image_source_cats = []
for image in all_images:
    image = image.get('src')
    image_source_cats.append(image)
    
image_source_cats = [image for image in image_source_cats if image.startswith('https')]

In [ ]:
#Find pictures of dogs...
browser.get_current_page()
browser.select_form()
#browser.get_current_form().print_summary()

search_term = 'Hund'

browser['q'] = search_term
browser.launch_browser()
response = browser.submit_selected()
new_url = browser.get_url()
browser.open(new_url)
page = browser.get_current_page()
all_images = page.find_all('img')

#... and save them into a list.
image_source_dogs = []
for image in all_images:
    image = image.get('src')
    image_source_dogs.append(image)
    
image_source_dogs = [image for image in image_source_dogs if image.startswith('https')]

In [ ]:
path = os.getcwd()
dst_dir_wbscrpng = os.path.join(path + '/catsordogs')
os.makedirs(dst_dir_wbscrpng, exist_ok=True)

#save cats

c = 0

for image in image_source_cats:
    save_as = os.path.join(dst_dir_wbscrpng, 'catordog' + str(c) + '.jpg')
    wget.download(image, save_as)
    c += 1

#save dogs   

d = c

for image in image_source_dogs:
    save_as = os.path.join(dst_dir_wbscrpng, 'catordog' + str(d) + '.jpg')
    wget.download(image, save_as)
    d += 1

In [ ]:
#Make and print predictions

images_to_predict = os.listdir('./catsordogs')

for i in images_to_predict: 
    img = tf.keras.utils.load_img(dst_dir_wbscrpng + "/" +i, target_size=(192,192))

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Add the image to a batch where it's the only member.

    prediction = model.predict_on_batch(img_array).flatten() #Predict on batch fügt eine Dimension hinzu, welche durch flatten wieder gelöscht werden soll.

    prediction = tf.where(prediction < 0.5, 0, 1) #das ist die threshold. 50% = 0, 50% = 1.
    
    print(
        "This image most likely belongs to {}:"
        .format(class_names[prediction[0]])
        )
    #making sure every image is printed with it's prediction.
    plt.imshow(img)
    plt.show()
    time.sleep(0.01)
